# 🧭 Capstone Project: The Learning Compass
### 5-Day AI Agents Intensive

**Project Overview**
This project implements a multi-agent system designed to generate personalized educational content. It utilizes the **Google Agent Development Kit (ADK)** to orchestrate a team of three specialized AI agents:

1.  **The Planner (Orchestrator):** Retrieves structured curriculum templates using a custom tool.
2.  **The Content Creator:** Generates detailed lessons based on the Planner's output.
3.  **The Evaluator:** Acts as "LLM-as-a-Judge" to quality-check the generated content.

**Technical Stack:**
* **Library:** `google-adk` (Agent Development Kit)
* **Model:** `gemini-2.5-flash-lite`
* **Pattern:** Sequential Multi-Agent Workflow with Tool Use

## Phase 1: Environment Setup & Tool Definition

In this phase, we initialize the environment with the necessary ADK libraries and authentication.

Crucially, we define our custom tool: `get_syllabus_template`.
* **Role:** This Python function acts as a retrieval tool (mocking a database) to fetch standard curriculum structures.
* **ADK Integration:** We use Python type hints (`str`, `dict`) and docstrings. The ADK uses these to automatically generate the schema that the Gemini model needs to understand how and when to call the tool.

In [1]:
# --- 1. Install ADK (If not already installed) ---
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 7.1 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.1

In [2]:

# --- 1. Imports & Setup ---
import os
from google.genai import types

# ADK Imports x
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner

# --- 2. Authentication (Environment Agnostic) ---
GOOGLE_API_KEY = None

# Try getting key from Kaggle Secrets (Cloud)
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    print("✅ Authenticated via Kaggle Secrets.")
except ImportError:
    # Fallback to Local .env file (Local Machine)
    try:
        from dotenv import load_dotenv
        load_dotenv()
        if "GOOGLE_API_KEY" in os.environ:
            GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
            print("✅ Authenticated via local .env file.")
    except ImportError:
        pass

if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
else:
    print("⚠️ Authentication Error: Could not find Kaggle Secrets or local .env")

✅ Authenticated via Kaggle Secrets.


In [3]:
def get_syllabus_template(difficulty_level: str) -> dict:
    """
    Retrieves a standard 3-day syllabus structure based on difficulty.
    
    Args:
        difficulty_level: The difficulty level ('Beginner', 'Intermediate', 'Advanced').
        
    Returns:
        A dictionary containing the day-by-day syllabus structure.
    """
    # Mock database logic
    templates = {
        "Beginner": {
            "Day 1": "Core Concepts & Definitions",
            "Day 2": "Basic Syntax & Usage",
            "Day 3": "Simple Practical Example"
        },
        "Intermediate": {
            "Day 1": "Advanced Data Structures",
            "Day 2": "Optimization Techniques",
            "Day 3": "Integration Patterns"
        },
        "Advanced": {
            "Day 1": "Theoretical Limits & Edge Cases",
            "Day 2": "Architecture Design Patterns",
            "Day 3": "Contributing to Source/Research"
        }
    }
    
    # Default to Beginner if input is weird
    result = templates.get(difficulty_level, templates["Beginner"])
    return {"status": "success", "syllabus": result}

print("✅ Tool defined successfully.")

✅ Tool defined successfully.


## Phase 2: Architecting the Agent Team

Here we define our three distinct agents using the `LlmAgent` class. We configure them with the `gemini-2.5-flash-lite` model and standard retry logic to handle potential API transient errors.

* **Agent 1: PlannerAgent**
    * **Goal:** Break down a broad topic into a 3-day plan.
    * **Tools:** Has exclusive access to `get_syllabus_template` to ensure structural consistency.
* **Agent 2: ContentAgent**
    * **Goal:** Write the actual educational content.
    * **Context:** Receives the specific plan from Agent 1.
* **Agent 3: EvaluatorAgent**
    * **Goal:** Quality assurance. It grades the output of Agent 2 on clarity and relevance.

In [4]:

# --- Configure standard retry logic ---
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# --- AGENT 1: THE PLANNER ---
# We use 'gemini-2.5-flash-lite' which is standard for this course environment
planner_agent = LlmAgent(
    name="PlannerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are an expert curriculum planner.
    1. Receive a topic and difficulty level from the user.
    2. Use the `get_syllabus_template` tool to get the structure.
    3. Return a concrete 3-day syllabus specific to the user's topic based on that template.
    """,
    tools=[get_syllabus_template] 
)

# --- AGENT 2: THE CONTENT CREATOR ---
content_agent = LlmAgent(
    name="ContentAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are an expert educational content writer.
    Your input will be a specific Day 1 topic from a syllabus.
    Write a detailed, engaging lesson (under 300 words).
    Include one code snippet or practical example.
    """
)

# --- AGENT 3: THE QUIZ MASTER (Evaluator) ---
evaluator_agent = LlmAgent(
    name="EvaluatorAgent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are a strict academic evaluator.
    Review the lesson content and output your grading in strictly valid JSON format.
    
    JSON Schema:
    {
        "score": int, // 1-5
        "reasoning": "string",
        "status": "PASS" | "FAIL" // Pass if score >= 4
    }
    """
)

print("✅ All 3 Agents initialized with correct model (gemini-2.5-flash-lite).")

✅ All 3 Agents initialized with correct model (gemini-2.5-flash-lite).


## Phase 3: Orchestration & Execution

We implement a **Sequential Workflow** to chain our agents together. Since the ADK returns a stream of events (including function calls, tool outputs, and text), simply grabbing the last item can be fragile.

To make this robust, we implement a helper function `get_last_text_response` that:
1.  Iterates backwards through the event history.
2.  Filters out tool calls and empty blocks.
3.  Returns the final text response from the model.

The `run_learning_compass` function then acts as the main loop, passing the output of one agent as the input context for the next.

In [5]:
import asyncio
from google.adk.runners import InMemoryRunner

# --- ROBUST TEXT EXTRACTION HELPER ---
def get_last_text_response(events):
    """
    Safely iterates backwards through events to find the last text response.
    Skips tool calls and empty events.
    """
    if not events:
        return "Error: No events returned from agent."
        
    # Iterate backwards to find the last model text response
    for event in reversed(events):
        try:
            # Check if the event has content and parts
            if event.content and event.content.parts:
                part = event.content.parts[0]
                
                # Check if this part is actually text (and not a function call)
                if hasattr(part, 'text') and part.text:
                    return part.text
        except Exception:
            continue
            
    return "Error: No text response found (Agent might have only called a tool)."

# ---  ORCHESTRATOR ---
async def run_learning_compass(user_topic, difficulty="Beginner"):
    print(f"🏁 Starting Learning Compass for: {user_topic} ({difficulty})")
    
    # 1. RUN PLANNER
    print("\n--- 🤖 PLANNER AGENT WORKING ---")
    planner_runner = InMemoryRunner(agent=planner_agent)
    planner_response = await planner_runner.run_debug(
        f"Create a syllabus for '{user_topic}' at {difficulty} level."
    )
    
    # USE HELPER HERE
    syllabus_text = get_last_text_response(planner_response)
    print(f"\n[Syllabus Output]:\n{syllabus_text}")
    
    if "Error" in syllabus_text:
        print("⚠️ Stopping early due to planner error.")
        return

    # 2. RUN CONTENT CREATOR
    print("\n--- 🤖 CONTENT AGENT WORKING ---")
    content_runner = InMemoryRunner(agent=content_agent)
    content_response = await content_runner.run_debug(
        f"Write the 'Day 1' lesson for this syllabus:\n{syllabus_text}"
    )
    
    # USE HELPER HERE
    lesson_text = get_last_text_response(content_response)
    print(f"\n[Lesson Output]:\n{lesson_text}")
    
    # 3. RUN EVALUATOR
    print("\n--- 🤖 EVALUATOR AGENT WORKING ---")
    eval_runner = InMemoryRunner(agent=evaluator_agent)
    eval_response = await eval_runner.run_debug(
        f"Evaluate this lesson content:\n{lesson_text}"
    )
    
    # USE HELPER HERE
    grade_text = get_last_text_response(eval_response)
    print(f"\n[Grading Output]:\n{grade_text}")


In [6]:
# --- EXECUTE ---
topic = input("Hey! I'm your learning compass. What do you want to learn?: ")
level = input("Which level (Beginner, Intermediate, Advanced) would you like?: ")
await run_learning_compass(topic, level)

Hey! I'm your learning compass. What do you want to learn?:  Data Structures and Algorithms
Which level (Beginner, Intermediate, Advanced) would you like?:  Intermediate


🏁 Starting Learning Compass for: Data Structures and Algorithms (Intermediate)

--- 🤖 PLANNER AGENT WORKING ---

 ### Created new session: debug_session_id

User > Create a syllabus for 'Data Structures and Algorithms' at Intermediate level.


PlannerAgent > In order to create a syllabus for 'Data Structures and Algorithms' at an 'Intermediate' level, I need to know what topics you'd like to cover. The template provides a structure, but I need the specific content. Can you please provide the topics for each of the 3 days?

[Syllabus Output]:
In order to create a syllabus for 'Data Structures and Algorithms' at an 'Intermediate' level, I need to know what topics you'd like to cover. The template provides a structure, but I need the specific content. Can you please provide the topics for each of the 3 days?

--- 🤖 CONTENT AGENT WORKING ---

 ### Created new session: debug_session_id

User > Write the 'Day 1' lesson for this syllabus:
In order to create a syllabus for 'Data Structures and Algorithms' at an 'Intermediate' level, I need to know what topics you'd like to cover. The template provides a structure, but I need the specific content. Can you please provide the topics for each of the 3 days?
ContentAgent > ## Day 1: Intr

## Project Reflection

### Architecture & Design
This project demonstrates a **sequential multi-agent architecture**. By separating concerns into Planning, Creation, and Evaluation, we avoid "context overloading" a single model. The Planner ensures the curriculum follows a proven structure (via the Tool), while the Evaluator acts as a quality gate.

### Key Challenges & Solutions
* **Tool Schemas:** Integrating Python functions with LLMs requires precise schemas. Using the ADK's auto-schema generation via docstrings simplified this significantly compared to writing raw JSON schemas.
* **Event Handling:** The ADK returns complex event streams (FunctionCall -> FunctionResponse -> ModelResponse). My initial implementation failed when trying to parse tool events as text. I resolved this by implementing a robust extractor function that specifically hunts for text parts in the event history.

### Future Improvements
To make this production-ready, I would:
1.  Replace the mock dictionary tool with a real vector database search (RAG) for curriculum retrieval.
2.  Implement a "Human-in-the-loop" checkpoint before the Content Agent runs, allowing the user to approve the syllabus.